# Reporting withholding tax separately for cash dividend corporate actions

In this Notebook we'll see how to use LUSID to perform the following task:

**<div align="center">As a portfolio manager, I want LUSID to automatically calculate the correct rate of withholding tax for corporate actions representing cash dividends in different jurisdictions, and report the tax due as separate cash balances.</div>**

In [1]:
# Set up LUSID
import os
import pandas as pd
import json
from IPython.core.display import HTML
from datetime import datetime, timedelta
import logging
logging.basicConfig(level=logging.INFO)

import lusid as lu
import lusid.api as la
import lusid.models as lm

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.lpt.lpt import to_date

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.11027.0,0.5.3205,"{'relation': 'RequestLogs', 'href': 'http://ja..."


In [2]:
# Create a scope and code to segregate data in this Notebook from others
module_scope = "corporate-actions"
module_code = "tax-tutorial"
print(f"'{module_scope}\{module_code}' scope and code created.")

'corporate-actions\tax-tutorial' scope and code created.


In [3]:
# Build all the required APIs
try:
    instruments_api = api_factory.build(la.InstrumentsApi)
    property_definition_api = api_factory.build(la.PropertyDefinitionsApi)
    corporate_action_sources_api = api_factory.build(la.CorporateActionSourcesApi)
    transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)
    portfolios_api = api_factory.build(la.PortfoliosApi)
    taxruleset_api = api_factory.build(la.TaxRuleSetsApi)
    transaction_config_api = api_factory.build(la.TransactionConfigurationApi)
    print("All APIs built correctly")
except lu.ApiException as e:
    print(e)

All APIs built correctly


## 1. Master instruments and attaching tax-related properties

Here we master BP and Microsoft as equity instruments in a custom instrument scope. Note GBP and USD currency instruments are pre-mastered in LUSID.

### 1.1 Create a property type for tax domicile

We'll use properties of this type (in conjunction with a similar portfolio property) to trigger a tax rule set to determine the correct rate of withholding tax for cash dividends in different jurisdictions.

In [4]:
# Create convenience function for creating property types
def create_property_type(property_domain, property_scope, property_code, data_type):
    
    # Define property type with a scope and code unique to the domain
    property_type_request = lm.CreatePropertyDefinitionRequest(
        domain = property_domain,
        scope = property_scope,
        code = property_code,
        display_name = property_code,
        data_type_id = lm.ResourceId(scope = "system", code = data_type)
    )
    
    # Create property type in LUSID
    try:
        property_type_response = property_definition_api.create_property_definition(
            create_property_definition_request = property_type_request
        )
        print(f"Property type created with the following key: {property_type_response.key}")
        return property_type_response.key
    except lu.ApiException as e:
        if json.loads(e.body)["name"] == "PropertyAlreadyExists":
            logging.info(
                f"Property type with the following key already exists: {property_type_request.domain}/{property_type_request.scope}/{property_type_request.code}"
            )  
        return f"{property_type_request.domain}/{property_type_request.scope}/{property_type_request.code}"

# Create a property type representing the tax domicile of an instrument, and capture the 3-stage key
instrument_tax_property_key = create_property_type("Instrument", "Tax", "Country", "string")

INFO:root:Property type with the following key already exists: Instrument/Tax/Country


### 1.2 Master instruments and attach tax domicile property values

In [5]:
# Create convenience function for mastering securities as equity instruments in a custom instrument scope
def master_instrument(figi, security, currency, domicile):
    
    # Define equity instrument
    instrument_request = {
        security: lm.InstrumentDefinition(
            name = security,
            identifiers = {"Figi": lm.InstrumentIdValue(value = figi)},
            definition = lm.Equity(instrument_type = "Equity", dom_ccy = currency),
            # Attach a property with a value declaring the tax domicile of the instrument
            properties = [
                lm.ModelProperty(
                    key = instrument_tax_property_key,
                    value = lm.PropertyValue(
                        label_value = domicile
                    )
                )
            ]
        )
    }
    
    # Upsert into LUSID
    instrument_response = instruments_api.upsert_instruments(
        request_body = instrument_request,
        scope = f"{module_scope}{module_code}"
    )

    # Transform upsert response to a dataframe and show internally-generated LUID identifier and tax-related property
    instrument_response_df = lusid_response_to_data_frame(list(instrument_response.values.values()))
    display(instrument_response_df[["name", "lusid_instrument_id", "properties.0.key", "properties.0.value.label_value"]])

# Master BP with a tax-related property value of UK    
master_instrument("BBG000C05BD1", "BP", "GBP", "UK")

# Master Microsoft with a tax-related property value of USA
master_instrument("BBG000BPH459", "Microsoft", "USD", "USA")

,name,lusid_instrument_id,properties.0.key,properties.0.value.label_value
0,BP,LUID_00003DLJ,Instrument/Tax/Country,UK


,name,lusid_instrument_id,properties.0.key,properties.0.value.label_value
0,Microsoft,LUID_00003DLK,Instrument/Tax/Country,USA


## 2. Create a corporate action source and load corporate actions

### 2.1 Create a corporate action source

In [6]:
# Define corporate action source
ca_source_request = lm.CreateCorporateActionSourceRequest(
    scope = module_scope,
    code = module_code,
    display_name = f"{module_scope}/{module_code} CA Source",
    # Attempt to resolve corporate actions to instruments in the custom scope before falling back to the default scope
    instrument_scopes = [f"{module_scope}{module_code}"]
)

# Create in LUSID
try:
    corporate_action_sources_api.create_corporate_action_source(
        create_corporate_action_source_request = ca_source_request
    )
    print(f"{module_scope}/{module_code} CA Source created")
except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a Corporate Action Source with id tax-tutorial because it already exists in scope corporate-actions.


### 2.2 Load corporate actions representing cash dividends

A corporate action has two important dates:

* `ex_date`: This is the date LUSID automatically applies the corporate action to portfolios that subscribe and already have a holding in the underlying instrument.
* `payment_date`: This is the date LUSID settles the corporate action.

In [7]:
# Create convenience function for loading corporate actions into a corporate action source
def load_ca(figi, action_date, ccy, out_units_factor, out_cost_factor):
    
    # Define corporate action
    ca_request = lm.UpsertInstrumentEventRequest(
        # A corporate action must have a unique identifier within its corporate action source
        instrument_event_id = f"{figi}-{action_date}-{ccy}",
        instrument_identifiers = {"Instrument/default/Figi": figi },
        description = f"{figi}-{action_date}-{ccy}",
        instrument_event = lm.TransitionEvent(
            # Use the generic transition event for now
            instrument_event_type = "TransitionEvent",
            # LUSID automatically applies the corporate action on this date
            ex_date = to_date(action_date).isoformat(),
            # LUSID automatically settles the corporate action on this date
            payment_date = (to_date(action_date) + timedelta(days = 4)).isoformat(),
             # These dates are informational only
            announcement_date = (to_date(action_date) - timedelta(days = 5)).isoformat(),
            record_date = to_date(action_date).isoformat(),
            # For each share held in the underlying instrument...
            input_transition = lm.InputTransition(
                units_factor = 1,
                cost_factor = 0
            ),
            output_transitions = [
                lm.OutputTransition(
                    # ...apply the units factor and cost factor to a particular currency instrument
                    instrument_identifiers = {"Instrument/default/Currency": ccy },
                    units_factor = out_units_factor,
                    cost_factor = out_cost_factor,
                    # Shouldn't be necessary: https://finbourne.atlassian.net/browse/IBOR-332
                    lusid_instrument_id = f"CCY_{ccy}",
                    instrument_scope = "default"
                )
            ]
        )
    )
    
    # Upsert corporate actions to LUSID as instrument events
    try:
        ca_response = corporate_action_sources_api.upsert_instrument_events(
            scope = module_scope,
            code = module_code,
            upsert_instrument_event_request = [ca_request]
        )
        # Response shouldn't be None: https://finbourne.atlassian.net/browse/IBOR-368
        # print(ca_response)
        print(f"Corporate action for FIGI {figi} loaded successfully")
    except lu.ApiException as e:
        print(e)

# Load BP cash dividend of 10 pence per share
load_ca("BBG000C05BD1", "2023-03-06", "GBP", 0.1, 0)

# Load Microsoft cash dividend of 10 cents per share
load_ca("BBG000BPH459", "2023-03-06", "USD", 0.1, 0)

Corporate action for FIGI BBG000C05BD1 loaded successfully
Corporate action for FIGI BBG000BPH459 loaded successfully


### 2.3 Confirm corporate action details

In [8]:
corp_actions_response = corporate_action_sources_api.get_instrument_events(scope = module_scope, code = module_code)
corp_actions_df = lusid_response_to_data_frame(corp_actions_response.values).transpose()
display(corp_actions_df)

,0,1
instrument_event_id,BBG000BPH459-2023-03-06-USD,BBG000C05BD1-2023-03-06-GBP
corporate_action_source_id.scope,corporate-actions,corporate-actions
corporate_action_source_id.code,tax-tutorial,tax-tutorial
instrument_identifiers.Instrument/default/Figi,BBG000BPH459,BBG000C05BD1
lusid_instrument_id,LUID_00003DLK,LUID_00003DLJ
instrument_scope,corporate-actionstax-tutorial,corporate-actionstax-tutorial
description,BBG000BPH459-2023-03-06-USD,BBG000C05BD1-2023-03-06-GBP
event_date_range.start,2023-03-06 00:00:00+00:00,2023-03-06 00:00:00+00:00
event_date_range.end,2023-03-10 00:00:00+00:00,2023-03-10 00:00:00+00:00
instrument_event.announcement_date,2023-03-01 00:00:00+00:00,2023-03-01 00:00:00+00:00


## 3. Set up a transaction portfolio

We need to:

* Subscribe to the corporate action source so corporate actions are applied to this portfolio.
* Attach a tax-related property to trigger a tax rule set  to determine the correct rate of withholding tax for cash dividends in different jurisdictions, in conjunction with similar instrument properties.
* Register a sub-holding key (SHK) so we can report withholding tax as a separate cash holding.


### 3.1 Create a property type for the portfolio's SHK

In [9]:
# Create a SHK that enables us to report withholding tax as a separate holding, and capture the 3-stage key
sub_holding_key = create_property_type("Transaction", "Tax", "CADividendWithholdingTax", "string")

INFO:root:Property type with the following key already exists: Transaction/Tax/CADividendWithholdingTax


### 3.2 Create a property type for the portfolio's tax domicile

In [10]:
# Create a property type representing the tax domicile of a portfolio, and capture the 3-stage key
portfolio_tax_property_key = create_property_type("Portfolio", "Tax", "Domicile", "string")

INFO:root:Property type with the following key already exists: Portfolio/Tax/Domicile


### 3.3 Create the portfolio

In [11]:
# Define transaction portfolio
portfolio_request=lm.CreateTransactionPortfolioRequest(
    display_name = f"Portfolio for corporate action tax tutorial",
    code = f"{module_code}",
    # Set the portfolio currency
    base_currency = "GBP",
    # Must be before first transaction recorded
    created = "2023-01-01",
    # Attempt to resolve transactions to instruments in the custom scope before falling back to the default scope
    instrument_scopes = [f"{module_scope}{module_code}"],
    # Subscribe to the corporate action source
    corporate_action_source_id=lm.ResourceId(
        scope = module_scope,
        code = module_code
    ),
    # Register the SHK with the portfolio
    sub_holding_keys = [sub_holding_key],
    # Attach a portfolio property with a value signifying the tax domicile
    properties={
        portfolio_tax_property_key: lm.ModelProperty(
            key = portfolio_tax_property_key,
            value = lm.PropertyValue(
                label_value = "GB"
            )
        )
    }
)

# Create transaction portfolio in LUSID
try:
    portfolio_response=transaction_portfolios_api.create_portfolio(
        scope = module_scope,
        create_transaction_portfolio_request = portfolio_request
    )
    # Confirm success
    print(f"Portfolio with display name '{portfolio_response.display_name}' created effective {str(portfolio_response.created)}")
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PortfolioWithIdAlreadyExists":
            logging.info(json.loads(e.body)["title"])

INFO:root:Could not create a portfolio with id 'tax-tutorial' because it already exists in scope 'corporate-actions'.


### 3.4 Confirm portfolio details

In [12]:
transaction_portfolio_response = transaction_portfolios_api.get_details(scope = module_scope, code = module_code)
tp_df = lusid_response_to_data_frame(transaction_portfolio_response)
# Drop some noisy indices
tp_df[~(tp_df.index.str.startswith('links') | tp_df.index.str.startswith('version') | tp_df.index.str.startswith('href'))]

,response_values
origin_portfolio_id.scope,corporate-actions
origin_portfolio_id.code,tax-tutorial
base_currency,GBP
corporate_action_source_id.scope,corporate-actions
corporate_action_source_id.code,tax-tutorial
sub_holding_keys.0,Transaction/Tax/CADividendWithholdingTax
instrument_scopes.0,corporate-actionstax-tutorial
accounting_method,Default
amortisation_method,NoAmortisation


### 3.5 Confirm the portfolio's tax-related property

The `GetDetails` API (above) has no `propertyKeys` parameter, but we can use the dedicated `GetPortfolioProperties` API instead.

In [13]:
tp_property_response = portfolios_api.get_portfolio_properties(scope = module_scope, code = module_code)
tp_property_response_df = lusid_response_to_data_frame(list(tp_property_response.properties.values()))
tp_property_response_df

,key,value.label_value,effective_from,effective_until
0,Portfolio/Tax/Domicile,GB,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00


### 3.6 Set initial positions in the portfolio

We'll set positions effective 2 January so the portfolio has holdings in the underlying equity instruments before the `ex-date` of the corporate actions.

In [14]:
# Create convenience function for adjusting holdings
def adjust_equity_holdings(figi, quantity, ccy):
    
    adjust_holdings_response = transaction_portfolios_api.batch_adjust_holdings(
        scope = module_scope,
        code = module_code,
        success_mode = "Atomic",
        request_body = {
            f"{figi}{quantity}{ccy}": lm.AdjustHoldingForDateRequest(
                effective_at = "2023-01-02",
                instrument_identifiers = {"Instrument/default/Figi": figi},
                tax_lots = [
                    lm.TargetTaxLotRequest(
                        units = quantity,
                        # Specify arbitrary price and cost
                        price = 2,
                        cost = lm.CurrencyAndAmount(amount = quantity * 2, currency = ccy),
                        portfolio_cost = quantity * 2
                    )
                ]
            )
        }
    )
    if len(adjust_holdings_response.failed) > 0:
        print(adjust_holdings_response.failed)
    else:
        print(f"Adjustment succeeded for {figi}")
        
# Set starting position for BP
adjust_equity_holdings("BBG000C05BD1", 100000, "GBP")
# Set starting position for Microsoft
adjust_equity_holdings("BBG000BPH459", 100000, "USD")

def adjust_cash_holdings(quantity, ccy):
    
    adjust_holdings_response = transaction_portfolios_api.batch_adjust_holdings(
        scope = module_scope,
        code = module_code,
        success_mode = "Atomic",
        request_body = {
            f"{quantity}{ccy}": lm.AdjustHoldingForDateRequest(
                effective_at = "2023-01-02",
                instrument_identifiers = {"Instrument/default/Currency": ccy},
                tax_lots = [
                    lm.TargetTaxLotRequest(
                        units = quantity,
                        price = 1,
                        cost = lm.CurrencyAndAmount(amount = quantity, currency = ccy),
                        portfolio_cost = quantity
                    )
                ]
            )
        }
    )
    if len(adjust_holdings_response.failed) > 0:
        print(adjust_holdings_response.failed)
    else:
        print(f"Adjustment succeeded for {ccy}")

# Set starting position for GBP        
adjust_cash_holdings(20000, "GBP")
# Set starting position for USD
adjust_cash_holdings(20000, "USD")

Adjustment succeeded for BBG000C05BD1
Adjustment succeeded for BBG000BPH459
Adjustment succeeded for GBP
Adjustment succeeded for USD


### 3.7 Confirm positions

In [15]:
# Create convenience function for generating a holdings report
def get_portfolio_holdings(holdings_date):
    
    get_holdings_response = transaction_portfolios_api.get_holdings(
        scope = module_scope, 
        code = module_code,
        # Retrieve properties to make results more intuitive
        property_keys = ["Instrument/default/Name", portfolio_tax_property_key, instrument_tax_property_key],
        effective_at = to_date(holdings_date).isoformat()
    )
    
    # Transform API response to a Pandas dataframe and show it
    get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
    # Drop some noisy columns
    get_holdings_response_df.drop(columns=[
       "instrument_scope", "holding_type", "SourcePortfolioId(default-Properties)", "properties.Portfolio/Tax/Domicile.effective_until", "properties.Instrument/Tax/Country.effective_until", "SourcePortfolioScope(default-Properties)", "cost_portfolio_ccy.amount", "cost_portfolio_ccy.currency", "currency"], inplace=True)
    display(get_holdings_response_df)
    
get_portfolio_holdings("2023-01-02")

,instrument_uid,CADividendWithholdingTax(Tax-SubHoldingKeys),Name(default-Properties),Domicile(Tax-Properties),units,settled_units,cost.amount,cost.currency,holding_type_name,Country(Tax-Properties)
0,CCY_USD,<Not Classified>,USD,GB,"20,000.00","20,000.00","20,000.00",USD,Balance,NaN
1,CCY_GBP,<Not Classified>,GBP,GB,"20,000.00","20,000.00","20,000.00",GBP,Balance,NaN
2,LUID_00003DLK,<Not Classified>,Microsoft,GB,"100,000.00","100,000.00","200,000.00",USD,Position,USA
3,LUID_00003DLJ,<Not Classified>,BP,GB,"100,000.00","100,000.00","200,000.00",GBP,Position,UK


### 3.7 Audit output transactions

LUSID automatically generates 'output transactions' under-the-hood to implement holdings adjustments (and other economic activity such as corporate actions that are not explicitly-specified 'input transactions').

Here we use a window of 2-3 January to cover the period of the holdings adjustments; we should see 4 output transactions.

In [16]:
# Create convenience function for generating output transactions for a particular window
def get_output_transactions(start, end, transpose):
    
    output_transactions_response = transaction_portfolios_api.build_transactions(
        scope = module_scope, 
        code = module_code,
        transaction_query_parameters = lm.TransactionQueryParameters(
            start_date = to_date(start).isoformat(),
            end_date = to_date(end).isoformat()
        ),
        # Retrieve property to make results more intuitive
        property_keys = ["Instrument/default/Name"]
    )
    
    if transpose == "vertical":
        output_transactions_response_df = lusid_response_to_data_frame(output_transactions_response).transpose()
    else:
        output_transactions_response_df = lusid_response_to_data_frame(output_transactions_response)
        output_transactions_response_df.drop(columns = ["instrument_identifiers.Instrument/default/Currency"], inplace = True)
    display(output_transactions_response_df)
    
get_output_transactions("2020-01-02", "2023-01-03", "horizontal")

,transaction_id,type,description,instrument_scope,instrument_uid,transaction_date,settlement_date,units,transaction_amount,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_to_portfolio_rate,transaction_currency,properties.Transaction/default/TradeToPortfolioRate.key,properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value,properties.Transaction/Tax/CADividendWithholdingTax.key,properties.Transaction/Tax/CADividendWithholdingTax.value.label_value,properties.Instrument/default/Name.key,properties.Instrument/default/Name.value.label_value,source,transaction_status,entry_date_time,realised_gain_loss,instrument_identifiers.Instrument/default/LusidInstrumentId,properties.Transaction/default/ResultantHolding.key,properties.Transaction/default/ResultantHolding.value.metric_value.value
0,2023-01-02T00:00:00.0000000+00:00,AdjustmentIncrease,Increase Adjustment,default,CCY_USD,2023-01-02 00:00:00+00:00,2023-01-02 00:00:00+00:00,"20,000.00","20,000.00",1.00,Price,"20,000.00",USD,1.00,1.00,USD,Transaction/default/TradeToPortfolioRate,1.00,Transaction/Tax/CADividendWithholdingTax,<Not Classified>,Instrument/default/Name,USD,,Active,2023-03-23 13:41:26.229114+00:00,[],NaN,NaN,NaN
1,2023-01-02T00:00:00.0000000+00:00,AdjustmentIncrease,Increase Adjustment,default,CCY_GBP,2023-01-02 00:00:00+00:00,2023-01-02 00:00:00+00:00,"20,000.00","20,000.00",1.00,Price,"20,000.00",GBP,1.00,1.00,GBP,Transaction/default/TradeToPortfolioRate,1.00,Transaction/Tax/CADividendWithholdingTax,<Not Classified>,Instrument/default/Name,GBP,,Active,2023-03-23 13:41:26.229114+00:00,[],NaN,NaN,NaN
2,2023-01-02T00:00:00.0000000+00:00,AdjustmentIncrease,Increase Adjustment,corporate-actionstax-tutorial,LUID_00003DLK,2023-01-02 00:00:00+00:00,2023-01-02 00:00:00+00:00,"100,000.00","200,000.00",2.00,Price,"200,000.00",USD,1.00,1.00,USD,Transaction/default/TradeToPortfolioRate,1.00,Transaction/Tax/CADividendWithholdingTax,<Not Classified>,Instrument/default/Name,Microsoft,,Active,2023-03-23 13:41:26.229114+00:00,[],LUID_00003DLK,Transaction/default/ResultantHolding,"100,000.00"
3,2023-01-02T00:00:00.0000000+00:00,AdjustmentIncrease,Increase Adjustment,corporate-actionstax-tutorial,LUID_00003DLJ,2023-01-02 00:00:00+00:00,2023-01-02 00:00:00+00:00,"100,000.00","200,000.00",2.00,Price,"200,000.00",GBP,1.00,1.00,GBP,Transaction/default/TradeToPortfolioRate,1.00,Transaction/Tax/CADividendWithholdingTax,<Not Classified>,Instrument/default/Name,BP,,Active,2023-03-23 13:41:26.229114+00:00,[],LUID_00003DLJ,Transaction/default/ResultantHolding,"100,000.00"


## 4. Create a tax rule set to calculate different rates of withholding tax

### 4.1 Create a property type for the amount of tax due

In [17]:
# Create a property type representing the amount of WH tax due, and capture the 3-stage key
outputtransaction_taxdue_property_key = create_property_type("Transaction", "Tax", "AmountDue", "number")

INFO:root:Property type with the following key already exists: Transaction/Tax/AmountDue


### 4.2 Create a tax rule for UK and US rates of withholding tax

A tax rule set has:

* An `effective_at` (or start) date that must precede the `ex_date` of a corporate action for LUSID to automatically apply the tax rule to that corporate action.
* Any number of tax rules, each with any number of match criteria that are processed in order. Note values can be compared against properties in the `Instrument`, `Portfolio` and `InstrumentEvent` domains, and also SHKs in the `Transaction` domain if the `criterion_type` is set to `SubHoldingKeyValueEquals`.
* An 'output property' in the `Transaction` domain that stores the result; that is, the amount of tax due for a corporate action, calculated using the appropriate rate.

In [18]:
# Convenience function for creating a tax rule set
def create_taxruleset(rule_start_date, uk_tax_rate, us_tax_rate):
    
    # Define tax rule set
    tax_request = lm.TaxRuleSet(
        id = lm.ResourceId(
            scope = "WithholdingTax",
            code = "UKPortfolios"
        ),
        description = "Withholding tax for UK-domiciled portfolios",
        display_name = "Withholding tax",
        # Specify output property to store the amount of tax due
        output_property_key = outputtransaction_taxdue_property_key,
        # Create rule for UK withholding tax
        rules = [
            lm.TaxRule(
                name = "UKDividendTax",
                description = "Dividend tax that applies to UK equities",
                rate = uk_tax_rate,
                match_criteria = [
                    lm.PropertyValueEquals(
                        criterion_type = "PropertyValueEquals",
                        property_key = instrument_tax_property_key,
                        value = "UK" 
                    ),
                    lm.PropertyValueEquals(
                        criterion_type="PropertyValueEquals",
                        property_key = portfolio_tax_property_key,
                        value = "GB" 
                    )
                ],
            ),
            # Create rule for US withholding tax
            lm.TaxRule(
                name = "USDividendTax",
                description = "Dividend tax that applies to US equities",
                rate = us_tax_rate,
                match_criteria = [
                    lm.PropertyValueEquals(
                        criterion_type = "PropertyValueEquals",
                        property_key = instrument_tax_property_key,
                        value = "USA" 
                    ),
                    lm.PropertyValueEquals(
                        criterion_type = "PropertyValueEquals",
                        property_key = portfolio_tax_property_key,
                        value = "GB" 
                    )
                ],
            )
        ]
    )
    
    # Create tax rule set in LUSID
    try:
        tax_response = taxruleset_api.create_tax_rule_set(
            create_tax_rule_set_request = tax_request,
            effective_at = to_date(rule_start_date).isoformat()
        )
        print("Tax rule set created")
    except lu.ApiException as e:
        if json.loads(e.body)["name"] == "TaxRuleSetAlreadyExists":
            logging.info(
                "Tax rule set already exists."
            )
        
create_taxruleset("2023-01-01", 0.25, 0.1)

INFO:root:Tax rule set already exists.


### 4.3 Confirm tax rule set details

In [19]:
tax_response = taxruleset_api.list_tax_rule_sets()
tax_response_df = lusid_response_to_data_frame(tax_response.values).transpose()
tax_response_df

,0
id.scope,WithholdingTax
id.code,UKPortfolios
display_name,Withholding tax
description,Withholding tax for UK-domiciled portfolios
output_property_key,Transaction/Tax/AmountDue
rules.0.name,UKDividendTax
rules.0.description,Dividend tax that applies to UK equities
rules.0.rate,0.25
rules.0.match_criteria.0.property_key,Instrument/Tax/Country
rules.0.match_criteria.0.value,UK


## 5. Create a custom transation type to report withholding tax separately

By default, LUSID applies the built-in `Dividend` transaction type to corporate actions representing cash dividends. This has the economic impact of automatically:

* Adding the dividend payment to the main cash balance in the portfolio
* Subtracting the withholding tax from the same cash balance.

We want to override the second behavior, and report withholding tax as a separate cash balance.

### 5.1 Create a custom side

Here we set the `security`, `currency` and `rate` fields of the custom side to the same values as the built-in `Side2`.

The `units` and `amount` fields are set to the value of the tax rule set's output property, to capture the amount of withholding tax due.

In [20]:
# Define custom side
side_request = lm.SideDefinitionRequest(
    security = "Txn:SettleCcy",
    currency = "Txn:SettlementCurrency",
    rate = "Txn:TradeToPortfolioRate",
    units = outputtransaction_taxdue_property_key,
    amount = outputtransaction_taxdue_property_key,
)

# Create custom side in LUSID
try:
    side_response = transaction_config_api.set_side_definition(
        # Specify the name of the custom side
        side = "BreakOutWithholdingTaxAsSeparateCashHoldingSide",
        side_definition_request = side_request
    )
    side_df = lusid_response_to_data_frame(side_response)
    # Drop some noisy indices
    display(side_df[~(side_df.index.str.startswith('links'))])
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "InvalidParameterValue":
        logging.info("Side definition already exists.")

,response_values
side,BreakOutWithholdingTaxAsSeparateCashHoldingSide
security,Txn:SettleCcy
currency,Txn:SettlementCurrency
rate,Txn:TradeToPortfolioRate
units,Transaction/Tax/AmountDue
amount,Transaction/Tax/AmountDue
notional_amount,0


### 5.2 Create a custom transaction type using the custom side

The first movement replicates the behavior of the built-in `Dividend` transaction type, to add the dividend payment to the main cash balance.

The second movement maps the output property captured by the custom side to the portfolio's SHK, to report it as a separate cash holding.

To override `Dividend`, we must assign the `TransactionConfiguration/default/InstrumentEventType` system property to the custom transaction type, with the special value of `transitionEvent/Dividend`.

In [21]:
# Define custom transaction type
transaction_type_request = lm.TransactionTypeRequest(
    # Create an alias
    aliases = [
        lm.TransactionTypeAlias(          
            type = "BreakOutWithholdingTaxAsSeparateCashHoldingType",
            description = "Track withholding tax as separate cash holding",
            transaction_class = "TestDividend",
            transaction_roles = "AllRoles"
        )
    ],
    movements = [
        # Create a positive movement that increases the main cash balance by the dividend payment
        lm.TransactionTypeMovement(            
            movement_types = "CashCommitment",
            direction = 1,
            side = "Side1",
            name = "Add dividend to main cash balance",
        ),
        # Create a negative movement that uses the custom side
        lm.TransactionTypeMovement(            
            movement_types = "CashCommitment",
            direction = -1,
            side = "BreakOutWithholdingTaxAsSeparateCashHoldingSide",
            name = "Track withholding tax as separate cash holding",
            # Map the tax rule set's output property to the portfolio's SHK
            mappings = [
                lm.TransactionPropertyMappingRequest(
                    property_key = f"{sub_holding_key}",
                    set_to = "WithholdingTax",
                )
            ],
        )
    ],
    properties = {
        # Assign a special system property to override the default `Dividend` TT and use this custom TT
        "TransactionConfiguration/default/InstrumentEventType": lm.PerpetualProperty(
            key = "TransactionConfiguration/default/InstrumentEventType",
            value = lm.PropertyValue(
                label_value = "transitionEvent/Dividend"
            )
        )
    },
)

# Create custom transaction type in LUSID    
try:
    transaction_type_response = transaction_config_api.set_transaction_type(
        source = "default",
        # Specify the primary alias name (in this case, the name of the only alias in the transaction type)
        type = "BreakOutWithholdingTaxAsSeparateCashHoldingType",
        transaction_type_request = transaction_type_request
    )
    tt_df = lusid_response_to_data_frame(transaction_type_response)
    # Drop some noisy indices
    display(tt_df[~(tt_df.index.str.startswith('links'))])
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "TransactionTypeDuplication":
        logging.info(f"Transaction type already exists.")

,response_values
aliases.0.type,BreakOutWithholdingTaxAsSeparateCashHoldingType
aliases.0.description,Track withholding tax as separate cash holding
aliases.0.transaction_class,TestDividend
aliases.0.transaction_roles,AllRoles
aliases.0.is_default,False
movements.0.movement_types,CashCommitment
movements.0.side,Side1
movements.0.direction,1
movements.0.properties,{}
movements.0.mappings,[]


## 6. Generate a holdings report

### 6.1 On the ex-dividend date

LUSID automatically applies the corporate actions on this date but note the dividend payments and withholding tax amounts are of holding type `CashCommitment`.

In [22]:
get_portfolio_holdings("2023-03-06")

,instrument_uid,CADividendWithholdingTax(Tax-SubHoldingKeys),Name(default-Properties),Domicile(Tax-Properties),units,settled_units,cost.amount,cost.currency,holding_type_name,Country(Tax-Properties),transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Currency,transaction.instrument_scope,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/Tax/CADividendWithholdingTax.key,transaction.properties.Transaction/Tax/CADividendWithholdingTax.value.label_value,transaction.properties.Transaction/Tax/AmountDue.key,transaction.properties.Transaction/Tax/AmountDue.value.metric_value.value,transaction.source,transaction.entry_date_time,transaction.transaction_status
0,CCY_USD,<Not Classified>,USD,GB,"20,000.00","20,000.00","20,000.00",USD,Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,CCY_GBP,<Not Classified>,GBP,GB,"20,000.00","20,000.00","20,000.00",GBP,Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,LUID_00003DLK,<Not Classified>,Microsoft,GB,"100,000.00","100,000.00","200,000.00",USD,Position,USA,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,LUID_00003DLJ,<Not Classified>,BP,GB,"100,000.00","100,000.00","200,000.00",GBP,Position,UK,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,CCY_USD,<Not Classified>,USD,GB,"10,000.00",0.00,"10,000.00",USD,CashCommitment,NaN,BBG000BPH459-2023-03-06-USD,BreakOutWithholdingTaxAsSeparateCashHoldingType,CCY_USD,default,CCY_USD,2023-03-06 00:00:00+00:00,2023-03-10 00:00:00+00:00,"10,000.00",1.00,Price,0.00,USD,1.00,USD,Transaction/Tax/CADividendWithholdingTax,<Not Classified>,Transaction/Tax/AmountDue,"1,000.00",default,2023-03-21 12:00:15.928627+00:00,Active
5,CCY_USD,WithholdingTax,USD,GB,"-1,000.00",0.00,"-1,000.00",USD,CashCommitment,NaN,BBG000BPH459-2023-03-06-USD,BreakOutWithholdingTaxAsSeparateCashHoldingType,CCY_USD,default,CCY_USD,2023-03-06 00:00:00+00:00,2023-03-10 00:00:00+00:00,"10,000.00",1.00,Price,0.00,USD,1.00,USD,Transaction/Tax/CADividendWithholdingTax,<Not Classified>,Transaction/Tax/AmountDue,"1,000.00",default,2023-03-21 12:00:15.928627+00:00,Active
6,CCY_GBP,<Not Classified>,GBP,GB,"10,000.00",0.00,"10,000.00",GBP,CashCommitment,NaN,BBG000C05BD1-2023-03-06-GBP,BreakOutWithholdingTaxAsSeparateCashHoldingType,CCY_GBP,default,CCY_GBP,2023-03-06 00:00:00+00:00,2023-03-10 00:00:00+00:00,"10,000.00",1.00,Price,0.00,GBP,1.00,GBP,Transaction/Tax/CADividendWithholdingTax,<Not Classified>,Transaction/Tax/AmountDue,"2,500.00",default,2023-03-21 12:00:16.436650+00:00,Active
7,CCY_GBP,WithholdingTax,GBP,GB,"-2,500.00",0.00,"-2,500.00",GBP,CashCommitment,NaN,BBG000C05BD1-2023-03-06-GBP,BreakOutWithholdingTaxAsSeparateCashHoldingType,CCY_GBP,default,CCY_GBP,2023-03-06 00:00:00+00:00,2023-03-10 00:00:00+00:00,"10,000.00",1.00,Price,0.00,GBP,1.00,GBP,Transaction/Tax/CADividendWithholdingTax,<Not Classified>,Transaction/Tax/AmountDue,"2,500.00",default,2023-03-21 12:00:16.436650+00:00,Active


### 6.2 On the payment date

LUSID automatically settles the corporate actions on this date. The dividend payments update the main cash balances, but the withholding tax amounts are reported as separate holdings using the `WithholdingTax` SHK.

In [23]:
get_portfolio_holdings("2023-03-11")

,instrument_uid,CADividendWithholdingTax(Tax-SubHoldingKeys),Name(default-Properties),Domicile(Tax-Properties),units,settled_units,cost.amount,cost.currency,holding_type_name,Country(Tax-Properties)
0,CCY_USD,<Not Classified>,USD,GB,"30,000.00","30,000.00","30,000.00",USD,Balance,NaN
1,CCY_USD,WithholdingTax,USD,GB,"-1,000.00","-1,000.00","-1,000.00",USD,Balance,NaN
2,CCY_GBP,<Not Classified>,GBP,GB,"30,000.00","30,000.00","30,000.00",GBP,Balance,NaN
3,CCY_GBP,WithholdingTax,GBP,GB,"-2,500.00","-2,500.00","-2,500.00",GBP,Balance,NaN
4,LUID_00003DLK,<Not Classified>,Microsoft,GB,"100,000.00","100,000.00","200,000.00",USD,Position,USA
5,LUID_00003DLJ,<Not Classified>,BP,GB,"100,000.00","100,000.00","200,000.00",GBP,Position,UK


### 6.3 Audit output transactions

Here we can use a window covering the ex-dividend date and payment dates to see the 'output transactions' that LUSID  automatically generates to implement the two corporate actions.

Note the **type** is our custom transaction type.

In [24]:
get_output_transactions("2023-03-06", "2023-03-10", "vertical")

,0,1
transaction_id,BBG000BPH459-2023-03-06-USD,BBG000C05BD1-2023-03-06-GBP
type,BreakOutWithholdingTaxAsSeparateCashHoldingType,BreakOutWithholdingTaxAsSeparateCashHoldingType
description,Track withholding tax as separate cash holding,Track withholding tax as separate cash holding
instrument_identifiers.Instrument/default/Currency,CCY_USD,CCY_GBP
instrument_scope,default,default
instrument_uid,CCY_USD,CCY_GBP
transaction_date,2023-03-06 00:00:00+00:00,2023-03-06 00:00:00+00:00
settlement_date,2023-03-10 00:00:00+00:00,2023-03-10 00:00:00+00:00
units,"10,000.00","10,000.00"
transaction_amount,"10,000.00","10,000.00"
